In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
51,11,Female,100.0,Strong,Kolkata,Yes
61,81,Female,98.0,Strong,Mumbai,No
5,84,Female,NaN,Mild,Bangalore,Yes
18,64,Female,98.0,Mild,Bangalore,Yes
3,31,Female,98.0,Mild,Kolkata,No


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('has_covid',axis=1),df['has_covid'],test_size=0.2)

In [6]:
X_train

,age,gender,fever,cough,city
14,51,Male,104.0,Mild,Bangalore
23,80,Female,98.0,Mild,Delhi
22,71,Female,98.0,Strong,Kolkata
86,25,Male,104.0,Mild,Bangalore
41,82,Male,NaN,Mild,Kolkata
...,...,...,...,...,...
82,24,Male,98.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
97,20,Female,101.0,Mild,Bangalore
98,5,Female,98.0,Strong,Mumbai


## Without Column Transformer

In [8]:
# adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [9]:
X_train_fever.shape

(80, 1)

In [11]:
#OrdinalEncoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [13]:
X_train_cough.shape

(80, 1)

In [15]:
#OneHotEncodinh -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [16]:
#extracting age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [17]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using ColumnTransformer

In [18]:
from sklearn.compose import ColumnTransformer

In [20]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse=False),['gender','city'])
],remainder='passthrough')

In [22]:
transformer.fit_transform(X_train).shape

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [23]:
transformer.fit_transform(X_test).shape

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)